In [43]:
import yfinance as yf
import quandl as ql
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [115]:
df = ql.get("WIKI/AMZN")

In [116]:
df

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1997-05-16,22.38,23.75,20.50,20.75,1225000.0,0.0,1.0,1.865000,1.979167,1.708333,1.729167,14700000.0
1997-05-19,20.50,21.25,19.50,20.50,508900.0,0.0,1.0,1.708333,1.770833,1.625000,1.708333,6106800.0
1997-05-20,20.75,21.00,19.63,19.63,455600.0,0.0,1.0,1.729167,1.750000,1.635833,1.635833,5467200.0
1997-05-21,19.25,19.75,16.50,17.13,1571100.0,0.0,1.0,1.604167,1.645833,1.375000,1.427500,18853200.0
1997-05-22,17.25,17.38,15.75,16.75,981400.0,0.0,1.0,1.437500,1.448333,1.312500,1.395833,11776800.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-21,1586.45,1590.00,1563.17,1581.86,4667291.0,0.0,1.0,1586.450000,1590.000000,1563.170000,1581.860000,4667291.0
2018-03-22,1565.47,1573.85,1542.40,1544.10,6177737.0,0.0,1.0,1565.470000,1573.850000,1542.400000,1544.100000,6177737.0
2018-03-23,1539.01,1549.02,1495.36,1495.56,7843966.0,0.0,1.0,1539.010000,1549.020000,1495.360000,1495.560000,7843966.0


In [121]:
df = df[['Adj. Close']]

In [123]:
# A variable for predicting 'n' days out into the future
forecast_out = 30

df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)

In [124]:
print(df.tail(5))

            Adj. Close  Prediction
Date                              
2018-03-21     1581.86         NaN
2018-03-22     1544.10         NaN
2018-03-23     1495.56         NaN
2018-03-26     1555.86         NaN
2018-03-27     1497.05         NaN


In [125]:
# Creating independent data set (X)
# Df to numpy array
X = np.array(df.drop(['Prediction'], axis=1))

# Remove the last 'n' rows

X = X[:-forecast_out]

print(X)

[[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1350.47      ]
 [1338.99      ]
 [1386.23      ]]


In [126]:
y = np.array(df['Prediction'])

y = y[:-forecast_out]

In [127]:
print(y)

[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [128]:
# Split the data into 80% training and 20% testing

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [129]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [130]:
svm_confidence = svr_rbf.score(x_test, y_test)
print(f"svm confidence: {svm_confidence}")

svm confidence: 0.9476463119730165


In [131]:
# Create and train the Linear Regression Model
lr = LinearRegression()

# Train the model
lr.fit(x_train, y_train)

LinearRegression()

In [132]:
lr_confidence = lr.score(x_test, y_test)
print(f"lr confidence: {lr_confidence}")

lr confidence: 0.989105911752871


In [57]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'], axis=1))[-forecast_out:]
print(x_forecast)

[[189.83999634]
 [189.86999512]
 [191.03999329]
 [192.3500061 ]
 [190.8999939 ]
 [186.88000488]
 [189.97999573]
 [189.99000549]
 [190.28999329]
 [191.28999329]
 [192.25      ]
 [194.02999878]
 [194.3500061 ]
 [195.86999512]
 [194.47999573]
 [196.88999939]
 [193.11999512]
 [207.1499939 ]
 [213.07000732]
 [214.24000549]
 [212.49000549]
 [216.66999817]
 [214.28999329]
 [209.67999268]
 [207.49000549]
 [208.13999939]
 [209.07000732]
 [213.25      ]
 [214.1000061 ]
 [210.61999512]]


In [58]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

print(f"--------------------------------------------------------------")

# Print support vector regression model predictions for the next 'n' days
svr_prediction = svr_rbf.predict(x_forecast)
print(svr_prediction)

[186.76408524 186.77949963 187.38068455 188.05381376 187.30874809
 185.24314079 186.83602169 186.84116505 186.995309   187.50914307
 188.00242722 188.91705124 189.0814819  189.86250404 189.148275
 190.38661698 188.44946035 195.6585517  198.70045628 199.3016412
 198.40243158 200.55025422 199.32732663 196.95855126 195.83326124
 196.16725025 196.64512001 198.79294265 199.22970474 197.44155654]
--------------------------------------------------------------
[179.25563743 179.28167314 181.78837639 184.58433709 181.30434221
 178.29940916 179.38457375 179.39466152 179.78227987 182.7079383
 184.70461163 174.9223211  173.80375621 181.14423921 173.63460431
 187.69593337 181.10058681 168.78386404 168.78929747 168.78929748
 168.78929745 168.78929748 168.78929748 168.78925889 168.78624957
 168.78836223 168.78915199 168.78929748 168.78929748 168.78929324]
